In [1]:
from datasets import load_dataset

'''
['med_qa_en_source', 'med_qa_en_bigbio_qa', 'med_qa_en_4options_source', 'med_qa_en_4options_bigbio_qa', 'med_qa_zh_source', 'med_qa_zh_bigbio_qa', 'med_qa_zh_4options_source', 'med_qa_zh_4options_bigbio_qa', 'med_qa_tw_source', 'med_qa_tw_bigbio_qa', 'med_qa_tw_en_source', 'med_qa_tw_en_bigbio_qa', 'med_qa_tw_zh_source', 'med_qa_tw_zh_bigbio_qa']
'''

# 加载MedQA数据集
dataset = load_dataset(
    'fzkuji/med_qa',
    'med_qa_en_4options_bigbio_qa',
    # 'train',
    trust_remote_code=True,
)

Using the latest cached version of the dataset since fzkuji/med_qa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'med_qa_en_4options_bigbio_qa' at /Users/fzkuji/.cache/huggingface/datasets/fzkuji___med_qa/med_qa_en_4options_bigbio_qa/0.0.0/6baf8bfacb0809793095b41610dee03fd6eeb698 (last modified on Mon Sep  2 14:18:16 2024).


In [2]:
# 访问训练集和测试集
train_dataset = dataset['train']
test_dataset = dataset['test']
validation_dataset = dataset['validation']

In [3]:
# 查看数据集的大小
print(len(train_dataset))
print(len(test_dataset))
print(len(validation_dataset))

10178
1273
1272


In [4]:
print(train_dataset[0])

{'meta_info': 'step2&3', 'question': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'answer_idx': 'E', 'answer': 'Nitrofurantoin', 'options': [{'key': 'A', 'value': 'Ampicillin'}, {'key': 'B', 'value': 'Ceftriaxone'}, {'key': 'C', 'value': 'Ciprofloxacin'}, {'key': 'D', 'value': 'Doxycycline'}, {'key': 'E', 'value': 'Nitrofurantoin'}]}


## 数据集预处理用于llama-factory

生成QA的prompt

In [3]:
from datasets import load_dataset
import os
import json

# Choose Language
language = "zh"  # Change this to 'en' or 'tw' for English or Traditional Chinese

# Load the dataset
dataset = load_dataset("fzkuji/med_qa", f"med_qa_{language}_4options_source", trust_remote_code=True)

# Define the save path
save_path = f"data/medical/MedQA/{language}/qa"  # Change this path to your local directory
os.makedirs(save_path, exist_ok=True)

# Function to save data as JSON with specified columns
def save_as_json(data, filename):
    file_path = os.path.join(save_path, filename)
    
    # Modify the data to include only 'question' and 'answer' columns
    data_to_save = [{
        "instruction": "Assuming you are a doctor, answer questions based on the patient's symptoms.",
        "input": item['question'],
        "output": item['answer']
    } for item in data]
    
    # Write the modified data to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

# Save the modified data for train, validation, and test splits
save_as_json(dataset['train'], 'train.json')
save_as_json(dataset['validation'], 'validation.json')
save_as_json(dataset['test'], 'test.json')

Generating train split:   0%|          | 0/27400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3426 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3425 [00:00<?, ? examples/s]

生成答案是文本的Multiple Choice的prompt（考虑了多种语言的格式）

In [16]:
from datasets import load_dataset
import os
import json

# Choose Language
language = "tw"  # Change this to 'en', 'zh' or 'tw' for English, Simplified Chinese or Traditional Chinese

# Load the dataset
dataset = load_dataset("fzkuji/med_qa", f"med_qa_{language}_bigbio_qa", trust_remote_code=True)

# Define the save path
save_path = f"data/medical/MedQA/{language}/multiple-choice"  # Change this path to your local directory
os.makedirs(save_path, exist_ok=True)

# Function to save data as JSON with specified columns
def save_as_json(data, filename):
    file_path = os.path.join(save_path, filename)
    
    # Modify the data to include only 'question' and 'answer' columns
    if language == 'en':
        data_to_save = [{
            "instruction": "Assuming you are a doctor, answer the following multiple-choice question based on the patient's symptoms.",
            "input": f"Question: {item['question']}\nOptions:\n" + "\n".join([f"\t{chr(65+i)}. {choice}" for i, choice in enumerate(item['choices'])]),
            "output": item['answer'][0]  # Assuming answer is a list, and you want the first element
        } for item in data]
    elif language == 'zh':
        data_to_save = [{
            "instruction": "假设您是一名医生，请根据患者的症状回答以下选择题。",
            "input": f"问题：{item['question']}\n选项：\n" + "\n".join([f"\t{chr(65+i)}. {choice}" for i, choice in enumerate(item['choices'])]),
            "output": item['answer'][0]  # Assuming answer is a list, and you want the first element
        } for item in data]
    elif language == 'tw':
        data_to_save = [{
            "instruction": "假设您是一名医生，请根据患者的症状回答以下选择题。",
            "input": f"問題：{item['question']}\n選項：\n" + "\n".join([f"\t{chr(65+i)}. {choice}" for i, choice in enumerate(item['choices'])]),
            "output": item['answer'][0]  # Assuming answer is a list, and you want the first element
        } for item in data]
    else:
        raise ValueError(f"Language '{language}' is not supported. Please choose 'en', 'zh' or 'tw'.")
    
    # Write the modified data to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

# Save the modified data for train, validation, and test splits
save_as_json(dataset['train'], 'train.json')
save_as_json(dataset['validation'], 'validation.json')
save_as_json(dataset['test'], 'test.json')

生成答案是ABCD的Multiple Choice的prompt（考虑了多种语言的格式，使用4options）

In [3]:
from datasets import load_dataset
import os
import json

# Choose Language
language = "en"  # Change this to 'en' or 'zh' for English or Simplified Chinese

# Load the dataset
dataset = load_dataset("fzkuji/med_qa", f"med_qa_{language}_4options_bigbio_qa", trust_remote_code=True)

# Define the save path
save_path = f"data/medical/MedQA/{language}/multiple-choice"  # Change this path to your local directory
os.makedirs(save_path, exist_ok=True)

# Mapping from index to letter
index_to_letter = {0: "A", 1: "B", 2: "C", 3: "D"}

# Function to save data as JSON with specified columns
def save_as_json(data, filename):
    file_path = os.path.join(save_path, filename)
    
    # Modify the data to include 'question', 'choices', and 'answer' columns
    if language == 'en':
        data_to_save = [{
            "instruction": "Assuming you are a doctor, answer the following multiple-choice question based on the patient's symptoms. Please select the correct option and only output the corresponding letter (A, B, C, or D).",
            "input": f"Question: {item['question']}\nOptions:\n" + "\n".join([f"\t{chr(65+i)}. {choice}." for i, choice in enumerate(item['choices'])]),
            "output": index_to_letter[item['choices'].index(item['answer'][0])]  # Convert the correct answer to A, B, C, or D
        } for item in data]
    elif language == 'zh':
        data_to_save = [{
            "instruction": "假设您是一名医生，请根据患者的症状回答以下选择题。请您选出正确的选项，并只输出对应的字母（A、B、C或D）。",
            "input": f"问题：{item['question']}\n选项：\n" + "\n".join([f"\t{chr(65+i)}. {choice}。" for i, choice in enumerate(item['choices'])]),
            "output": index_to_letter[item['choices'].index(item['answer'][0])]  # Convert the correct answer to A, B, C, or D
        } for item in data]
    else:
        raise ValueError(f"Language '{language}' is not supported. Please choose 'en', 'zh' or 'tw'.")
    
    # Write the modified data to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

# Save the modified data for train, validation, and test splits
save_as_json(dataset['train'], 'train.json')
save_as_json(dataset['validation'], 'validation.json')
save_as_json(dataset['test'], 'test.json')


Using the latest cached version of the dataset since fzkuji/med_qa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'med_qa_en_4options_bigbio_qa' at /Users/fzkuji/.cache/huggingface/datasets/fzkuji___med_qa/med_qa_en_4options_bigbio_qa/0.0.0/6baf8bfacb0809793095b41610dee03fd6eeb698 (last modified on Mon Sep  2 14:18:16 2024).


生成答案是原始文本的Multiple Choice的prompt（考虑了多种语言的格式，使用4options）

In [6]:
from datasets import load_dataset
import os
import json

# Choose Language
language = "zh"  # Change this to 'en' or 'zh' for English or Simplified Chinese

# Load the dataset
dataset = load_dataset("fzkuji/med_qa", f"med_qa_{language}_4options_bigbio_qa", trust_remote_code=True)

# Define the save path
save_path = f"data/medical/MedQA/{language}/multiple-choice"  # Change this path to your local directory
os.makedirs(save_path, exist_ok=True)

# Function to save data as JSON with specified columns
def save_as_json(data, filename):
    file_path = os.path.join(save_path, filename)
    
    # Modify the data to include 'question', 'choices', and 'answer' columns
    if language == 'en':
        data_to_save = [{
            "instruction": "Assuming you are a doctor, answer the following multiple-choice question based on the patient's symptoms. Please select the correct option and only output the corresponding letter (A, B, C, or D).",
            "input": f"Question: {item['question']}\nOptions:\n" + "\n".join([f"\t{chr(65+i)}. {choice}." for i, choice in enumerate(item['choices'])]),
            "output": item['answer'][0]  # Convert the correct answer to A, B, C, or D
        } for item in data]
    elif language == 'zh':
        data_to_save = [{
            "instruction": "假设您是一名医生，请根据患者的症状回答以下选择题。请您输出答案的文本内容（不包含选项序号）。",
            "input": f"问题：{item['question']}\n选项：\n" + "\n".join([f"\t{chr(65+i)}. {choice}。" for i, choice in enumerate(item['choices'])]),
            "output": item['answer'][0]  # Convert the correct answer to A, B, C, or D
        } for item in data]
    else:
        raise ValueError(f"Language '{language}' is not supported. Please choose 'en', 'zh' or 'tw'.")
    
    # Write the modified data to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

# Save the modified data for train, validation, and test splits
save_as_json(dataset['train'], 'train.json')
save_as_json(dataset['validation'], 'validation.json')
save_as_json(dataset['test'], 'test.json')


Using the latest cached version of the dataset since fzkuji/med_qa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'med_qa_zh_4options_bigbio_qa' at /Users/fzkuji/.cache/huggingface/datasets/fzkuji___med_qa/med_qa_zh_4options_bigbio_qa/0.0.0/6baf8bfacb0809793095b41610dee03fd6eeb698 (last modified on Mon Sep  2 14:06:48 2024).
